---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [10]:
def graph_identification():
    
#     for graph in P1_Graphs:
#         print(nx.average_clustering(graph))
#         print(nx.average_shortest_path_length(graph))
#         print(nx.info(graph))
#         print()
#         print()
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [11]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [147]:
def salary_predictions():
    
    from sklearn.linear_model import LogisticRegression
    
    df = pd.DataFrame(index=G.nodes())
    df['Dept'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['deg_cen'] = pd.Series(nx.degree_centrality(G))
    df['btw_cen'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False))
    df['SPR'] = pd.Series(nx.pagerank(G, alpha=0.8))
    df['hub'] = pd.Series(nx.hits(G)[0])
    df['auth'] = pd.Series(nx.hits(G)[1])
    df['Salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    X_test = df[pd.isnull(df['Salary'])].drop('Salary', axis=1)
    X_train = df.dropna()

    y_train = X_train['Salary'].values
    X_train = X_train.drop('Salary', axis=1)
    
    lr = LogisticRegression(C=100).fit(X_train, y_train)
    
    preds = lr.predict_proba(X_test)[:, 1]
    
    return pd.Series(preds, index=X_test.index)

salary_predictions()

1       0.152589
2       0.560334
5       0.958367
8       0.123888
14      0.339417
18      0.173532
27      0.248915
30      0.280660
31      0.171882
34      0.096994
37      0.116855
40      0.227361
45      0.182345
54      0.214164
55      0.173516
60      0.184366
62      0.992338
65      0.445504
77      0.068601
79      0.124534
97      0.088752
101     0.100771
103     0.169803
108     0.133205
113     0.427223
122     0.068754
141     0.417730
142     0.895969
144     0.103886
145     0.424256
          ...   
913     0.075303
914     0.078459
915     0.036768
918     0.078413
923     0.041614
926     0.068502
931     0.073050
934     0.066427
939     0.035402
944     0.047155
945     0.040262
947     0.045552
950     0.231688
951     0.054345
953     0.060284
959     0.031767
962     0.036458
963     0.158138
968     0.044116
969     0.041306
974     0.039604
984     0.064324
987     0.066094
989     0.038154
991     0.062510
992     0.033959
994     0.045250
996     0.0391

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [154]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [192]:
def new_connections_predictions():
    
    from sklearn.linear_model import LogisticRegression
    
    com_nei = {i: len(list(nx.common_neighbors(G, i[0], i[1]))) for i in nx.non_edges(G)}
    jac_coef = {(i[0], i[1]): i[2] for i in nx.jaccard_coefficient(G)}
    res_all = {(i[0], i[1]): i[2] for i in nx.resource_allocation_index(G)}
    future_connections.sort_index(inplace=True)
    
    df = pd.DataFrame(index=nx.non_edges(G))
    df['Common N.'] = pd.Series(com_nei)
    df['Jac Coef'] = pd.Series(jac_coef)
    df['Res All'] = pd.Series(res_all)
    df['Fut Con'] = pd.Series(future_connections['Future Connection'])
    
    X_test = df[df['Fut Con'].isnull()].drop('Fut Con', axis=1)
    X_train = df.dropna()
    
    y_train = X_train['Fut Con'].values
    X_train = X_train.drop('Fut Con', axis=1)
    
    lr = LogisticRegression(C=100).fit(X_train, y_train)
    
    preds = lr.predict_proba(X_test)[:, 1]
    
    return pd.Series(preds, index=X_test.index)

new_connections_predictions()

(0, 9)          0.038264
(0, 19)         0.091788
(0, 20)         0.314841
(0, 35)         0.017151
(0, 38)         0.010527
(0, 42)         0.795407
(0, 43)         0.020565
(0, 50)         0.010527
(0, 53)         0.636963
(0, 54)         0.058252
(0, 62)         0.476088
(0, 63)         0.069501
(0, 69)         0.027439
(0, 72)         0.010527
(0, 83)         0.264647
(0, 90)         0.034291
(0, 91)         0.018299
(0, 95)         0.156371
(0, 100)        0.037601
(0, 106)        0.948425
(0, 108)        0.017706
(0, 109)        0.010527
(0, 110)        0.010527
(0, 118)        0.010527
(0, 122)        0.036689
(0, 131)        0.056690
(0, 133)        0.789440
(0, 140)        0.086060
(0, 149)        0.256702
(0, 151)        0.010527
                  ...   
(988, 989)      0.010527
(988, 996)      0.010527
(988, 997)      0.540180
(988, 1000)     0.010527
(988, 1002)     0.010527
(989, 994)      0.010527
(989, 996)      0.010527
(989, 1003)     0.010527
(989, 1004)     0.010527
